In [ ]:
import tensorflow as tf

print(tf.__version__)

from tensorflow.python.keras import layers
import matplotlib.pyplot as plt
import os
from random import random
import numpy as np
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from tensorflow import keras

from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
# from tensorflow_addons.layers.normalization.instancenormalization import InstanceNormalization

In [ ]:
from tensorflow.python.client import device_lib
print(tf.test.gpu_device_name())
print(device_lib.list_local_devices())

In [ ]:
# GENERATOR ARCHITECTURE: https://towardsdatascience.com/overview-of-cyclegan-architecture-and-training-afee31612a2f

def create_downsample_layers(model):
    # 64x7x7 conv, stride 1
    model.add(layers.Conv2D(64, (7, 7), strides=(1, 1), padding="SAME", input_shape=[256, 256, 3]))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    assert model.output_shape == (None, 256, 256, 64)
    
    #128x3x3 conv, stride 2
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    assert model.output_shape == (None, 128, 128, 128)
    
    # 256x3x3 conv, stride 2
    model.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    assert model.output_shape == (None, 64, 64, 256)

def create_residual_block(model):
    # 256x3x3 conv
    model.add(layers.Conv2D(256, (3, 3),padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(256, (3, 3),padding="SAME"))
    model.add(layers.BatchNormalization())
    
def create_residual_blocks(model, num_blocks):
    for i in range(num_blocks):
        create_residual_block(model)
        
def create_upsample_layers(model):
    # 128x3x3 convTranspose, stride 2
    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    assert model.output_shape == (None, 128, 128, 128)
    
    # 64x3x3 convTranspose, stride 2
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    assert model.output_shape == (None, 256, 256, 64)
    
    # 3x7x7 conv, stride 1
    model.add(layers.Conv2D(3, (7, 7), strides=(1, 1), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation(tf.keras.activations.tanh))
    assert model.output_shape == (None, 256, 256, 3)

def create_generator():
    model = tf.keras.models.Sequential()
    
    create_downsample_layers(model)
    assert model.output_shape == (None, 64, 64, 256)
    
    create_residual_blocks(model, 9)
    assert model.output_shape == (None, 64, 64, 256)
    
    create_upsample_layers(model)
    assert model.output_shape == (None, 256, 256, 3)
    
    return model

In [ ]:
assert create_generator()

In [ ]:
def create_discriminator():
    model = tf.keras.models.Sequential()
    
    # 64x4x4 conv, stride 2
    model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding="SAME", input_shape=[256, 256, 3]))
    model.add(layers.LeakyReLU(alpha=0.2))
    assert model.output_shape == (None, 128, 128, 64)
    
    # 128x4x4 conv, stride 2
    model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    assert model.output_shape == (None, 64, 64, 128)
    
    # 256x4x4 conv, stride 2
    model.add(layers.Conv2D(256, (4, 4), strides=(2, 2), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    assert model.output_shape == (None, 32, 32, 256)
    
    # 512x4x4 conv, stride 2
    model.add(layers.Conv2D(512, (4, 4), strides=(2, 2), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    assert model.output_shape == (None, 16, 16, 512)
    
    # 512x4x4 conv, stride 1
    model.add(layers.Conv2D(512, (4, 4), padding="SAME"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    assert model.output_shape == (None, 16, 16, 512)
    
    # 1x4x4 conv, stride 1
    model.add(layers.Conv2D(1, (4, 4), strides=(1, 1), padding="SAME"))
    assert model.output_shape == (None, 16, 16, 1)
    
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
    
    return model

In [ ]:
assert create_discriminator()

In [ ]:
def create_composite_model(generator1, discriminator, generator2, image_shape):
    # through the composite model, we only want to train generator1
    generator1.trainable = True
    discriminator.trainable = False
    generator2.trainable = False
    
    img_same = tf.keras.Input(shape=image_shape)
    img_opposite = tf.keras.Input(shape=image_shape)
    
    output_fake = generator1(img_opposite)
    output_dis = discriminator(output_fake)
    
    output_identity = generator1(img_same)
    
    output_forward = generator2(output_fake)
    
    output_fake_gen2 = generator2(img_same)
    output_backward = generator1(output_fake_gen2)
    
    model = tf.keras.Model([img_opposite, img_same], [output_dis, output_identity, output_forward, output_backward])
    opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
    return model

In [ ]:
def preprocess(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    img = (img - 127.5) / 127.5 
    img = tf.image.central_crop(img, 0.5)
    img = tf.image.resize(img, size=[256, 256])
    return img

In [ ]:
def load_data():
    animation_image_dir = './animated scenery'
#     subdirectory_names = []

#     for item in os.walk(animation_image_dir):
#         subdirectory_names = item[1]
#         break

#     animation_filenames = tf.data.Dataset.list_files(['./images/%s/*.jpg' % (s) for s in subdirectory_names])
    animation_filenames = tf.data.Dataset.list_files(animation_image_dir+'/*.jpg')
    animation_dataset = animation_filenames.map(lambda x: preprocess(x))
    
    flickr_filenames = tf.data.Dataset.list_files('./flickr/*.jpg')
    flickr_dataset = flickr_filenames.map(lambda x: preprocess(x))
        
    return (np.stack(list(animation_dataset)), np.stack(list(flickr_dataset)))

In [ ]:
def grab_real_samples(data, num_samples, shape):
    # print(data.shape[0])
    ix = randint(0, data.shape[0], num_samples)
    x = tf.convert_to_tensor(data[ix])
    y = tf.convert_to_tensor(ones((num_samples, shape, shape, 1))) # 1 = real img
    return x, y

In [ ]:
def grab_fake_samples(generator, data, shape):
    # print(data.shape)
    x = generator.predict(data)
    y = tf.convert_to_tensor(zeros((len(x), shape, shape, 1))) # 0 = fake img
    return x, y

In [ ]:
def save_models(step, g_anim_to_real, g_real_to_anim, dis1, dis2, com1, com2):
    filename1 = './models new/anim_to_real_%06d.h5' % (step+1)
    g_anim_to_real.save(filename1)

    filename2 = './models new/real_to_anim_%06d.h5' % (step+1)
    g_real_to_anim.save(filename2)
    
    filename3 = './models new/dis1%06d.h5' % (step+1)
    dis1.save(filename3)
    
    filename4 = './models new/dis2%06d.h5' % (step+1)
    dis2.save(filename4)
    
    filename5 = './models new/com1%06d.h5' % (step+1)
    com1.save(filename5)
    
    filename6 = './models new/com2%06d.h5' % (step+1)
    com2.save(filename6)
    
    print('Saved models: %s %s %s %s %s %s' % (filename1, filename2, filename3, filename4, filename5, filename6))

In [ ]:
def summarize(step, generator, train_x, name, num_samples=5):
    x_in, _ = grab_real_samples(train_x, num_samples, 0)
    x_out, _ = grab_fake_samples(generator, x_in, 0)

    x_in = (x_in + 1) / 2.0
    x_out = (x_out + 1) / 2.0
    
    for i in range(num_samples):
        plt.subplot(2, num_samples, 1 + i)
        plt.axis('off')
        plt.imshow(x_in[i])

    for i in range(num_samples):
        plt.subplot(2, num_samples, 1 + num_samples + i)
        plt.axis('off')
        plt.imshow(x_out[i])

    filename = './summaries new/%s_summary_%06d.png' % (name, (step+1))
    plt.savefig(filename)
    plt.close()

In [ ]:
def update_fake_pool(pool, imgs, max_size=50):
    selected = []
    for img in imgs:
        if len(pool) < max_size:
            pool.append(img)
            selected.append(img)
        elif random() < 0.5:
            selected.append(img)
        else:
            i = randint(0, len(pool))
            selected.append(pool[i])
            pool[i] = img
    return tf.convert_to_tensor(selected)

In [ ]:
def train(dis1, dis2, generator1, generator2, composite1, composite2, dataset, offset=0):
    EPOCHS = 100
    BATCHES = 1
    N_PATCH = dis1.output_shape[1]
    train1, train2 = dataset
    pool1, pool2 = list(), list()
    batches_per_epoch = int(min(len(train1), len(train2)) / BATCHES)
    num_steps = batches_per_epoch * EPOCHS
    
    for i in range(num_steps):
        # print("Step %d" % (i))
        x_real1, y_real1 = grab_real_samples(train1, BATCHES, N_PATCH)
        x_real2, y_real2 = grab_real_samples(train2, BATCHES, N_PATCH)
                
        x_fake1, y_fake1 = grab_fake_samples(generator2, x_real2, N_PATCH)
        x_fake2, y_fake2 = grab_fake_samples(generator1, x_real1, N_PATCH)
        
        x_fake1 = update_fake_pool(pool1, x_fake1)
        x_fake2 = update_fake_pool(pool2, x_fake2)
        
        # print("Training Composite 2")        
        gen2_loss, _, _, _, _ = composite2.train_on_batch([x_real2, x_real1], [y_real1, x_real1, x_real2, x_real1])
                                
        dis1_loss1 = dis1.train_on_batch(x_real1, y_real1)
        dis1_loss2 = dis1.train_on_batch(x_fake1, y_fake1)
                
        # print("Training Composite 1")
        
        gen1_loss, _, _, _, _ = composite1.train_on_batch([x_real1, x_real2], [y_real2, x_real2, x_real1, x_real2])
                
        dis2_loss1 = dis2.train_on_batch(x_real2, y_real2)
        dis2_loss2 = dis2.train_on_batch(x_fake2, y_fake2)
                
        print('Losses: %d, d1[%.3f,%.3f] d2[%.3f,%.3f] g[%.3f,%.3f]' % (i+1+offset, dis1_loss1,dis1_loss2, dis2_loss1,dis2_loss2, gen1_loss,gen2_loss))
        
        if (i+offset+1) % 300 == 0:
            summarize(i+offset, generator1, train1, 'Generator 1 crop')
            summarize(i+offset, generator2, train2, 'Generator 2 crop')
        if (i+offset+1) % 500 == 0:
            save_models(i+offset, generator1, generator2, dis1, dis2, composite1, composite2)

In [ ]:
dataset = load_data()
image_shape = None

x = dataset[0][0]
fig, ax = plt.subplots(ncols = 1)
ax.imshow((x * 127.5 + 127.5).astype(int))
plt.show()
image_shape = x.shape
    
y = dataset[1][0]
fig, ax = plt.subplots(ncols = 1)
ax.imshow((y * 127.5 + 127.5).astype(int))
plt.show()

In [ ]:
assert image_shape == (256, 256, 3)

In [ ]:
print(len(dataset[0]))

In [20]:
# generator1 = create_generator()
# generator2 = create_generator()
# dis1 = create_discriminator()
# dis2 = create_discriminator()
# composite1 = create_composite_model(generator1, dis2, generator2, image_shape)
# composite2 = create_composite_model(generator2, dis1, generator1, image_shape)

In [ ]:
# train(dis1, dis2, generator1, generator2, composite1, composite2, dataset)

Losses: 1, d1[1.427,1.606] d2[1.305,0.952] g[18.021,17.714]
Losses: 2, d1[3.109,1.134] d2[1.677,1.726] g[19.517,22.003]
Losses: 3, d1[18.167,10.030] d2[14.803,8.999] g[23.363,26.809]
Losses: 4, d1[2.559,3.834] d2[13.187,2.530] g[17.959,17.845]
Losses: 5, d1[2.302,0.507] d2[6.116,1.493] g[20.232,21.601]
Losses: 6, d1[7.710,0.537] d2[2.142,1.085] g[16.531,16.719]
Losses: 7, d1[3.890,0.260] d2[4.526,0.371] g[18.220,21.153]
Losses: 8, d1[3.447,0.272] d2[6.167,0.287] g[17.886,18.620]
Losses: 9, d1[2.868,0.146] d2[1.069,0.160] g[15.676,17.292]
Losses: 10, d1[1.245,0.155] d2[1.467,0.126] g[14.566,14.678]
Losses: 11, d1[1.523,0.176] d2[0.780,0.077] g[14.244,14.031]
Losses: 12, d1[0.462,0.234] d2[0.215,0.098] g[16.154,15.643]
Losses: 13, d1[0.591,0.242] d2[0.304,0.130] g[10.745,12.254]
Losses: 14, d1[0.535,0.176] d2[0.487,0.161] g[13.554,14.487]
Losses: 15, d1[0.374,0.112] d2[0.504,0.330] g[12.137,12.510]
Losses: 16, d1[0.250,0.099] d2[0.417,0.649] g[12.607,12.227]
Losses: 17, d1[0.239,0.052] d

In [ ]:
generator1 = keras.models.load_model('./models new/anim_to_real_063000.h5')
generator2 = keras.models.load_model('./models new/real_to_anim_063000.h5')
dis1 = keras.models.load_model('./models new/dis1063000.h5')
dis2 = keras.models.load_model('./models new/dis2063000.h5')

In [20]:
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [21]:
# composite1 = create_composite_model(generator1, dis2, generator2, image_shape)
# composite2 = create_composite_model(generator2, dis1, generator1, image_shape)

In [ ]:
y = dataset[1][0]
fig, ax = plt.subplots(ncols = 1)
ax.imshow((y * 127.5 + 127.5).astype(int))
plt.show()

x_pred = generator2(tf.expand_dims(y, 0))
fig, ax = plt.subplots(ncols = 1)
ax.imshow((x_pred[0] + 1)/2)
plt.show()

In [ ]:
# train(dis1, dis2, generator1, generator2, composite1, composite2, dataset, offset=63000)